In [0]:
# ---------- CNPJ (ZIP -> Volume -> Delta/UC) ----------
ensure_catalog_schema(raw_catalog, raw_schema_cnpj)

# garantir pastas no Volume
dbutils.fs.mkdirs(f"{vol_dbfs_base}/zips")
dbutils.fs.mkdirs(f"{vol_dbfs_base}/extract")

zip_files = [f for f in list_dir(src_cnpj_dir) if f.name.lower().endswith(".zip")]

for f in zip_files:
    base_no_ext = f.name[:-4]
    fqn_table   = f"`{raw_catalog}`.`{raw_schema_cnpj}`.`{base_no_ext}`"
    delta_dir   = f"{raw_base}/cnpj/{base_no_ext}"

    zip_dbfs_dst   = f"{vol_dbfs_base}/zips/{f.name}"
    zip_local_dst  = f"{vol_local_base}/zips/{f.name}"
    extract_local  = f"{vol_local_base}/extract/{base_no_ext}"
    extract_dbfs   = f"{vol_dbfs_base}/extract/{base_no_ext}"  # para spark.read

    print(f"\n==> ZIP: {f.path}")
    print("    staging zip  :", zip_dbfs_dst)
    print("    extract local:", extract_local)
    print("    delta        :", delta_dir)
    print("    tabela       :", fqn_table)

    # 1) Copiar ZIP para o Volume (staging)
    if dry_run:
        print(f"[DRY-RUN] cp {f.path} -> {zip_dbfs_dst}")
    else:
        dbutils.fs.cp(f.path, zip_dbfs_dst, recurse=False)

    # 2) Unzip dentro do Volume (POSIX /Volumes/…)
    if dry_run:
        print(f"[DRY-RUN] unzip {zip_local_dst} -> {extract_local}")
    else:
        # limpar extração anterior
        if os.path.exists(extract_local):
            shutil.rmtree(extract_local, ignore_errors=True)
        os.makedirs(extract_local, exist_ok=True)
        with zipfile.ZipFile(zip_local_dst, 'r') as z:
            z.extractall(extract_local)

    # 3) Ler CSVs extraídos (via dbfs:/Volumes/…) e gravar Delta em raw
    if dry_run:
        print(f"[DRY-RUN] ler CSV(s) de {extract_dbfs}/**/*.csv")
        print(f"[DRY-RUN] salvar Delta -> {delta_dir}")
        create_or_replace_external_delta_table(fqn_table, delta_dir)
    else:
        df = (spark.read
                    .options(**csv_options)
                    .option("recursiveFileLookup", "true")
                    .csv(f"{extract_dbfs}"))
        (df.write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .option("delta.columnMapping.mode", "name")
            .option("path", delta_dir)              # importante
            .saveAsTable(fqn_table))                # registra já com a propriedade